# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
from flask import Flask, render_template, request


### 2. Load the dataset

In [ ]:
url = "https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv"
df = pd.read_csv(url)

### 3. Check the shape and basic information of the dataset.

In [ ]:
print(df.shape)
print(df.info())

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [ ]:
df = df.drop_duplicates()

### 5. Drop the columns which you think redundant for the analysis.

In [ ]:
columns_to_drop = ['Car_Name', 'Present_Price', 'Owner']
df = df.drop(columns=columns_to_drop)

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [ ]:
df['age_of_the_car'] = 2023 - df['Year']
df = df.drop(columns=['Year'])

### 7. Encode the categorical columns

In [ ]:
label_encoder = LabelEncoder()
df['Fuel_Type'] = label_encoder.fit_transform(df['Fuel_Type'])
df['Seller_Type'] = label_encoder.fit_transform(df['Seller_Type'])
df['Transmission'] = label_encoder.fit_transform(df['Transmission'])

### 8. Separate the target and independent features.

In [ ]:
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

print("R2 Score for Train:", r2_score(y_train, y_train_pred))
print("R2 Score for Test:", r2_score(y_test, y_test_pred))


### 11. Create a pickle file with an extension as .pkl

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump(rf_model, file)


### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Used Cars Price Prediction</title>
</head>
<body>
    <h1>Used Cars Price Prediction</h1>
    <form action="{{ url_for('predict') }}" method="post">
        <label for="age_of_the_car">Age of the Car:</label>
        <input type="number" name="age_of_the_car" required>
        <br>
        <label for="kms_driven">Kilometers Driven:</label>
        <input type="number" name="kms_driven" required>
        <br>
        <label for="fuel_type">Fuel Type:</label>
        <select name="fuel_type" required>
            <option value="0">Petrol</option>
            <option value="1">Diesel</option>
            <option value="2">CNG</option>
        </select>
        <br>
        <label for="seller_type">


### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask, render_template, request
import pickle

app = Flask(__name__)

# Load the model
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    features = [float(x) for x in request.form.values()]
    prediction = model.predict([features])
    return render_template('index.html', prediction=f'Predicted Selling Price: {prediction[0]:.2f} lakhs')

if __name__ == '__main__':
    app.run(debug=True)

### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

### Happy Learning :)